<a href="https://colab.research.google.com/github/Chaalal-anfel/search-engine/blob/master/model_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing


installation des bibliotèque nécessaire !!!!
 

# Nouvelle section

2. importation et lecture de fichier de données:

In [51]:
df = pd.read_csv('donnees_vitesse_ch3_v1.6.csv')

In [52]:
print(df)


                   temps     id  vitesse_med  vitesse_max  distance
0    2022-03-18 14:50:11  94374            0            0         0
1    2022-03-18 14:50:00  94373           11           43      1633
2    2022-03-18 14:49:59  94372           11           43      1633
3    2022-03-18 14:48:00  94371           11           43      1633
4    2022-03-18 14:40:00  94370            0           22        37
..                   ...    ...          ...          ...       ...
167  2022-03-16 17:55:47  94207           59          118     49529
168  2022-03-16 17:50:00  94206           57          111     43448
169  2022-03-16 17:40:00  94205           49          110     30651
170  2022-03-16 17:30:00  94204           37          100     18234
171  2022-03-16 17:20:00  94203           35          100     12777

[172 rows x 5 columns]


3. description des colonnes du **dataframe** :

In [53]:
print('-'*50)
print("Columns in data")
print('-'*50)
print(df.columns.values)

--------------------------------------------------
Columns in data
--------------------------------------------------
['temps' 'id' 'vitesse_med' 'vitesse_max' 'distance']


In [54]:
df.head()

,temps,id,vitesse_med,vitesse_max,distance
0,2022-03-18 14:50:11,94374,0,0,0
1,2022-03-18 14:50:00,94373,11,43,1633
2,2022-03-18 14:49:59,94372,11,43,1633
3,2022-03-18 14:48:00,94371,11,43,1633
4,2022-03-18 14:40:00,94370,0,22,37


3. faire les modifications nécessaire 
   - changer le type de la colonne **temps** from string to datetime 

In [55]:
print(df['temps'])

0      2022-03-18 14:50:11
1      2022-03-18 14:50:00
2      2022-03-18 14:49:59
3      2022-03-18 14:48:00
4      2022-03-18 14:40:00
              ...         
167    2022-03-16 17:55:47
168    2022-03-16 17:50:00
169    2022-03-16 17:40:00
170    2022-03-16 17:30:00
171    2022-03-16 17:20:00
Name: temps, Length: 172, dtype: object


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   temps        172 non-null    object
 1   id           172 non-null    int64 
 2   vitesse_med  172 non-null    int64 
 3   vitesse_max  172 non-null    int64 
 4   distance     172 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 6.8+ KB


In [57]:
df['temps']= pd.to_datetime(df['temps'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   temps        172 non-null    datetime64[ns]
 1   id           172 non-null    int64         
 2   vitesse_med  172 non-null    int64         
 3   vitesse_max  172 non-null    int64         
 4   distance     172 non-null    int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 6.8 KB


4. calcule de la différence en terme de seconde de chaque deux lignes succéssives de la colonne temps :

In [58]:
df['temps'] = df['temps'].view(int).div(1e9).diff().fillna(0).abs()
# la fonction abs() sera utile si l'ordre de fichier est descendant par rapport au collone temps
df['temps']=df.temps.abs()
print(df)

      temps     id  vitesse_med  vitesse_max  distance
0       0.0  94374            0            0         0
1      11.0  94373           11           43      1633
2       1.0  94372           11           43      1633
3     119.0  94371           11           43      1633
4     480.0  94370            0           22        37
..      ...    ...          ...          ...       ...
167  1093.0  94207           59          118     49529
168   347.0  94206           57          111     43448
169   600.0  94205           49          110     30651
170   600.0  94204           37          100     18234
171   600.0  94203           35          100     12777

[172 rows x 5 columns]


5. calcule de l'acceleration en foction de la variation de la vitesse et le temps:
- calcule de la variation de vitesse entre chaque 2 point en fonction de la vitesse moyenne et la vitesse maximale
- calcule de l'accéleration en terme de deux type de vitesse en ma convertant en seconds


In [59]:
df['vitesse_med_dif'] = df['vitesse_med'].view(int).diff().fillna(0).abs()
df['vitesse_max_dif'] = df['vitesse_max'].view(int).diff().fillna(0).abs()
print(df)

      temps     id  vitesse_med  vitesse_max  distance  vitesse_med_dif  \
0       0.0  94374            0            0         0              0.0   
1      11.0  94373           11           43      1633             11.0   
2       1.0  94372           11           43      1633              0.0   
3     119.0  94371           11           43      1633              0.0   
4     480.0  94370            0           22        37             11.0   
..      ...    ...          ...          ...       ...              ...   
167  1093.0  94207           59          118     49529              0.0   
168   347.0  94206           57          111     43448              2.0   
169   600.0  94205           49          110     30651              8.0   
170   600.0  94204           37          100     18234             12.0   
171   600.0  94203           35          100     12777              2.0   

     vitesse_max_dif  
0                0.0  
1               43.0  
2                0.0  
3      

In [60]:
df['acc_vitesse_max']=df['vitesse_max_dif']*0.27778/df['temps']
df['acc_vitesse_med']=df['vitesse_med_dif']*0.27778/df['temps']
print(df)

      temps     id  vitesse_med  vitesse_max  distance  vitesse_med_dif  \
0       0.0  94374            0            0         0              0.0   
1      11.0  94373           11           43      1633             11.0   
2       1.0  94372           11           43      1633              0.0   
3     119.0  94371           11           43      1633              0.0   
4     480.0  94370            0           22        37             11.0   
..      ...    ...          ...          ...       ...              ...   
167  1093.0  94207           59          118     49529              0.0   
168   347.0  94206           57          111     43448              2.0   
169   600.0  94205           49          110     30651              8.0   
170   600.0  94204           37          100     18234             12.0   
171   600.0  94203           35          100     12777              2.0   

     vitesse_max_dif  acc_vitesse_max  acc_vitesse_med  
0                0.0              NaN     